In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier 
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from catboost import CatBoostClassifier

In [2]:
train = pd.read_csv("Train.csv")
test = pd.read_csv("Test.csv")
sample = pd.read_csv("SampleSubmission.csv")

In [3]:
train.head()

,ID,Year_of_Birth,Education_Level,Marital_Status,Disposable_Income,No_of_Kids_in_home,No_of_Teen_in_home,Date_Customer,Recency,Discounted_Purchases,...,Amount_on_SweetProducts,Amount_on_GoldProds,WebVisitsMonth,Cmp3Accepted,Cmp4Accepted,Cmp5Accepted,Cmp1Accepted,Cmp2Accepted,Any_Complain,Response
0,ID_4A9AR6FQ,1955,Graduation,Divorced,77504.4,1,1,22-06-2014,56,2,...,6,20,3,0,0,0,0,0,0,0
1,ID_X28T3VEK,1958,Graduation,Together,56784.0,0,1,01-08-2013,17,6,...,15,19,6,0,0,0,0,0,0,0
2,ID_AWXARH57,1962,Graduation,Single,103714.8,0,0,21-11-2013,17,0,...,18,37,2,0,0,0,1,0,0,1
3,ID_FQVZHE81,1979,2n Cycle,Single,46311.6,1,0,13-09-2013,49,3,...,24,24,8,0,0,0,0,0,0,0
4,ID_QVLWGPRN,1959,Graduation,Married,87486.0,0,0,21-01-2014,59,1,...,165,203,1,0,0,0,1,0,0,0


In [4]:
# adding the age column
train["Age"] = 2021 - train["Year_of_Birth"]
test["Age"] = 2021 - test["Year_of_Birth"]

# converting the date column to datetime
train["Date_Customer"] = pd.to_datetime(train["Date_Customer"])
test["Date_Customer"] = pd.to_datetime(test["Date_Customer"])

# creating a new column for the number of years of joined
train["no_of_years_joined"] = 2021- train["Date_Customer"].dt.year
test["no_of_years_joined"] = 2021- test["Date_Customer"].dt.year

train["avg_web_purchased"] = train["WebPurchases"] / train["no_of_years_joined"]
test["avg_web_purchased"] = test["WebPurchases"] / test["no_of_years_joined"]

train["month_joined"] = train["Date_Customer"].dt.month
test["month_joined"] =  test["Date_Customer"].dt.month

train["week_recency"] = train["Recency"] / 7
test["week_recency"] = test["Recency"] /7

In [5]:
train.head()

,ID,Year_of_Birth,Education_Level,Marital_Status,Disposable_Income,No_of_Kids_in_home,No_of_Teen_in_home,Date_Customer,Recency,Discounted_Purchases,...,Cmp5Accepted,Cmp1Accepted,Cmp2Accepted,Any_Complain,Response,Age,no_of_years_joined,avg_web_purchased,month_joined,week_recency
0,ID_4A9AR6FQ,1955,Graduation,Divorced,77504.4,1,1,2014-06-22,56,2,...,0,0,0,0,0,66,7,0.142857,6,8.000000
1,ID_X28T3VEK,1958,Graduation,Together,56784.0,0,1,2013-01-08,17,6,...,0,0,0,0,0,63,8,0.625000,1,2.428571
2,ID_AWXARH57,1962,Graduation,Single,103714.8,0,0,2013-11-21,17,0,...,0,1,0,0,1,59,8,0.875000,11,2.428571
3,ID_FQVZHE81,1979,2n Cycle,Single,46311.6,1,0,2013-09-13,49,3,...,0,0,0,0,0,42,8,0.500000,9,7.000000
4,ID_QVLWGPRN,1959,Graduation,Married,87486.0,0,0,2014-01-21,59,1,...,0,1,0,0,0,62,7,0.428571,1,8.428571


### Dropping some columns

In [6]:
train = train.drop(columns=["ID", "Date_Customer", "Year_of_Birth"], axis=1)
test = test.drop(columns=["ID", "Date_Customer", "Year_of_Birth"], axis=1)

### Fixing the missing value problem

In [7]:
from sklearn.impute import SimpleImputer

train_col = train.columns
test_col = test.columns

imputer = SimpleImputer(strategy="median")
train[["Disposable_Income"]] = pd.DataFrame(imputer.fit_transform(train[["Disposable_Income"]]))
test[["Disposable_Income"]] = pd.DataFrame(imputer.transform(test[["Disposable_Income"]]))

### Converting the categorical variables to numerical

In [8]:
# Target Encoding
from category_encoders import TargetEncoder
cols = ["Education_Level", "Marital_Status"]
def target_encode(cols, train_data, test_data=None):
    for col in cols:
        target_enc = TargetEncoder()
        train_data[col] = target_enc.fit_transform(train_data[col], train_data["Response"])
        if test_data is not None:
            test_data[col] = target_enc.transform(test_data[col])
target_encode(cols, train, test)

C:\Users\HP\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\HP\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [9]:
from sklearn import model_selection
train["stra_kfold"] = -1
train_y = train["Response"]
skf = model_selection.StratifiedKFold(n_splits=8, shuffle=True, random_state=42)
for fold, (train_indicies, valid_indicies) in enumerate(skf.split(train, train_y)):
    train.loc[valid_indicies, "stra_kfold"] = fold

In [10]:
train.to_csv("train_folds.csv", index=False)
test.to_csv("cleaned_test.csv", index=False)

In [11]:
train = pd.read_csv("train_folds.csv")
test = pd.read_csv("cleaned_test.csv")
sample = pd.read_csv("SampleSubmission.csv")

useful_features = [i for i in train.columns if i not in("stra_kfold", "Response")]

In [12]:
final_prediction = []
score = []
for fold in range(8):
    x_train = train[train.stra_kfold != fold].reset_index(drop=True)
    x_valid = train[train.stra_kfold == fold].reset_index(drop=True)
    df_test = test.copy()
    
    y_train = x_train.Response
    y_valid = x_valid.Response
    
    x_train = x_train[useful_features]
    x_valid = x_valid[useful_features]
    
    sc = StandardScaler()
    scaled_x_train = pd.DataFrame(sc.fit_transform(x_train))
    scaled_x_valid = pd.DataFrame(sc.transform(x_valid))
    scaled_test = pd.DataFrame(sc.transform(df_test))
    
    scaled_x_train.columns = x_train.columns
    scaled_x_valid.columns = x_valid.columns
    scaled_test.columns = df_test.columns
    
    
    model = CatBoostClassifier(random_state=fold, scale_pos_weight=5.56, verbose=0,n_estimators = 401,
    learning_rate = 0.1004261502680816,
    max_depth = 7)
    model.fit(scaled_x_train, y_train)
    preds = model.predict(scaled_x_valid)
    test_preds = model.predict(scaled_test)
    
    final_prediction.append(test_preds)
    scores = f1_score(y_valid,preds)
    score.append(scores)

print(np.mean(score))

0.580572489079955


In [13]:
final_preds = np.mean(np.column_stack(final_prediction),axis=1)
final_preds = [1 if i >=0.4 else 0 for i in final_preds]

sample.Response = final_preds
sample.to_csv("Submission20.csv", index=False)